## Jacob Roach

In [1]:
# Import the needed Packages.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM

## Data Collection and Feature Engineering
Before any modeling was performed, the necessary data was collected using two distinct platforms. The first data that was collected was Twitter data. This was done using the Twitter Developer API, as well as the `tweepy` module. Tweets containing the word "bitcoin" were streamed for several days. This data was written to a `.pkl` file, and saved for later feature engineering.

The other data that was collected was the value of a single Bitcoin. During the same interval (plus twenty-four hours after the last Tweet was recorded) that the Twitter data was collected, the value of a Bitcoin was recorded each minute, along with the corresponding time stamp.

Once the Twitter and Bitcoin data was recorded, further feature engineering was employed. For each Tweet stored, the corresponding price of Bitcoin at the time the Tweet was made was added as the `inital_price` for the Tweet. Then, for each Tweet, if the price of Bitcoin increased within twenty-four hours of the time the Tweet was made, the feature `increase` was assigned a value of `1`. Otherwise, `increase` is assigned the value of `0`.

Finally, for each Tweet recorded, the text of that Tweet is cleaned and standardized. This cleaned Tweet is then BERTified, and a vector of length 512 is returned. This vector is stored as the `bertified` feature. Only the `bertified` and `increase` features are kept, and these form the training data to be used in this notebook.

In [2]:
# Read in the training data.
data = pd.read_pickle("../data/training_data.pkl")

# Reset the index.
data = data.reset_index(drop=True)

# Convert each list to an array.
data["bertified"] = data["bertified"].apply(lambda x: np.asarray(x))

Once the training data has been read in, the data will be quickly inspected, to show the reader the nature of the dataset.

In [3]:
# Investigate the DataFrame.
print("There are", len(data), "rows in the DataFrame.")
print("There are", len(data.loc[data["increase"] == 1, ]), "records with an increase, and", len(data.loc[data["increase"] == 0, ]), "with a decrease.\n")

# Now, show the summary of the data.
print(data["increase"].describe())

There are 114233 rows in the DataFrame.
There are 55912 records with an increase, and 58321 with a decrease.

count    114233.000000
mean          0.489456
std           0.499891
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: increase, dtype: float64


Now that the training data has been loaded, it can be partitioned into training and testing sets.

In [4]:
# Create training and testing.
x_train, x_test, y_train, y_test = train_test_split(data["bertified"], data["increase"], test_size=.10)

# Conver to Tensors.
x_train = tf.convert_to_tensor(x_train.to_list())
y_train = tf.convert_to_tensor(y_train.to_list())
x_test = tf.convert_to_tensor(x_test.to_list())
y_test = tf.convert_to_tensor(y_test.to_list())

2022-03-21 19:47:09.456878: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Train the model.
input_layer = Input((512,))
dense = Dense(128, activation="relu")(input_layer)
output = Dense(2, activation="softmax")(dense)  # Output values is the number of classes.
model = Model(input_layer, output)

# Compile the model.
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="adam",
              metrics=["accuracy"])

# Fit the model.
model.fit(x_train, y_train, epochs=40, validation_data=(x_test, y_test))

Epoch 1/40
3213/3213 [==============================] - 6s 2ms/step - loss: 0.6719 - accuracy: 0.5587 - val_loss: 0.6612 - val_accuracy: 0.5755
Epoch 2/40
3213/3213 [==============================] - 5s 2ms/step - loss: 0.6454 - accuracy: 0.5994 - val_loss: 0.6468 - val_accuracy: 0.5927
Epoch 3/40
3213/3213 [==============================] - 5s 2ms/step - loss: 0.6255 - accuracy: 0.6252 - val_loss: 0.6442 - val_accuracy: 0.5955
Epoch 4/40
3213/3213 [==============================] - 5s 2ms/step - loss: 0.6076 - accuracy: 0.6473 - val_loss: 0.6425 - val_accuracy: 0.5996
Epoch 5/40
3213/3213 [==============================] - 5s 2ms/step - loss: 0.5910 - accuracy: 0.6648 - val_loss: 0.6489 - val_accuracy: 0.5989
Epoch 6/40
3213/3213 [==============================] - 5s 2ms/step - loss: 0.5756 - accuracy: 0.6804 - val_loss: 0.6505 - val_accuracy: 0.6028
Epoch 7/40
3213/3213 [==============================] - 5s 2ms/step - loss: 0.5607 - accuracy: 0.6949 - val_loss: 0.6574 - val_accuracy:

In [6]:
# Test the model.
model.evaluate(x=x_test, y=y_test)

357/357 [==============================] - 1s 1ms/step - loss: 0.9972 - accuracy: 0.6142


[0.9972397685050964, 0.6142331957817078]

In [7]:
# Check out the DataFrame.
data.head()

,bertified,increase
0,"[0.04822409898042679, -0.015833385288715363, 0...",1
1,"[0.08190881460905075, 0.03084167279303074, -0....",1
2,"[0.09046691656112671, -0.06422483175992966, 0....",1
3,"[0.03809173032641411, 0.014239655807614326, -0...",1
4,"[-0.049746137112379074, 0.010899393819272518, ...",1
